In [1]:
import numpy as np
import pandas as pd
import seaborn as sns  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# Pending

In [ ]:
# from scipy.io import loadmat

# link = r"D:\external_data\Experiment4"
# filename = r"\Dataset_PWR_WiFi.mat"
# directory = link + filename

# mat = loadmat(directory)

In [ ]:
# # import experimental dataset 2
# folderpath2 = "D:/external_data/Experiment3/csv_files/exp_2"  # CHANGE THIS IF THE PATH CHANGED
# df_exp2 = import_clean_data('exp2',folderpath2)

# lab

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from data.process_data import stacking
import matplotlib.pyplot as plt



def visual_spectrogram(img,title='spectrogram'):
    
    if len(img.shape) == 3:
        
        img.squeeze()
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(img)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
    plt.figure(figsize = (15,50))
    plt.imshow(img,cmap='jet')
    plt.title(title)
    plt.show()
    
    return 

In [ ]:
import torchvision.models as models
        
#visualize weights for alexnet - first conv layer
alexnet = models.alexnet(pretrained=False)

# Store all conv layer in alexnet
k = []
for m in alexnet.modules():
    if isinstance(m, nn.Conv2d):
        print(m.weight.data.shape)
        k.append(m)
    else:  
        continue


In [ ]:
def visual_single_filter(weight):
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(weight)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
#     plt.figure(figsize = (15,10))
    plt.imshow(weight,cmap='jet')
#     plt.show()
    
    return 

def visualise_filters(layer,figsize=(20,15)):
    
    weights = layer.weight.data

    if weights.shape[0] > weights.shape[1]:
        
        weights = weights.transpose(1,0)
        
    rows = weights.shape[0]
    
    cols = weights.shape[1]
    
    axes = []
    
    fig= plt.figure()

    for n in range(rows):

        for c in range(cols):
            
            axes.append(fig.add_subplot(rows, cols, n*cols+c+1))

            single_filter = weights[n,c]
        
            visual_single_filter(single_filter)
            
#     fig.tight_layout()    
    
    plt.show()     
    
    return 

visualise_filters(example)

# model factory

In [4]:
import time
from time import gmtime, strftime

In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

##### extra setting

In [3]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

torch.cuda.device_count()

torch.cuda.set_device(device)

cuda:0


In [3]:
np.random.seed(1024)
torch.manual_seed(1024)
# torch.set_deterministic(True)

NameError: name 'np' is not defined

### 0. Default

In [7]:
# model 
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        pass
    
    def forward(self,X):
        return X

### 1. Section

In [6]:
from models import ED_module, Lambda, Classifier
from torchsummary import summary

In [7]:
class Encoder_4(nn.Module):
    """
    Encoder for spectrogram (1,65,65), 4 layer
    """
    def __init__(self,num_filters):
        super(Encoder_4, self).__init__()
        l1,l2,l3,4 = num_filters
        ### 1st ###
        self.conv1 = nn.Conv2d(1,l1,kernel_size=5,stride=2)
        self.norm1 = nn.BatchNorm2d(l1)
        self.actv1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2))
        ### 2nd ###
        self.conv2 = nn.Conv2d(l1,l2,kernel_size=3,stride=2)
        self.norm2 = nn.BatchNorm2d(l2)
        self.actv2 = nn.ReLU()
        self.pool2 = Lambda(lambda x:x) # nn.MaxPool2d(kernel_size=(2,2))
        ### 3rd ###
        self.conv3 = nn.Conv2d(l2,l3,kernel_size=2,stride=2)
        self.norm3 = Lambda(lambda x:x)
        self.actv3 = nn.Tanh()
        self.pool3 = nn.AvgPool2d(kernel_size=(2,2))
        ### 4th ###
        self.conv3 = nn.Conv2d(l2,l3,kernel_size=2,stride=2)
        self.norm3 = Lambda(lambda x:x)
        self.actv3 = nn.Tanh()
        self.pool3 = nn.AvgPool2d(kernel_size=(2,2))
        
    def forward(self,X):
        X = self.pool1(self.actv1(self.norm1(self.conv1(X))))
        X = self.pool2(self.actv2(self.norm2(self.conv2(X))))
        X = self.pool3(self.actv3(self.norm3(self.conv3(X))))
        X = torch.flatten(X, 1)
        # print(X.shape)
        return X
    
# mdl = Encoder([64,128,512]).cuda()
# summary(mdl,(1,65,65),batch_size=128)

In [8]:
class Projection_head(nn.Module):
    """
    Projection head: 
    
    head: linear/mlp
    """
    def __init__(self,dim_in,feat_dim,head='linear'):
        super(Projection_head, self).__init__()
        if head == 'linear':
            self.head = nn.Linear(dim_in, feat_dim)
        elif head == 'mlp':
            self.head = nn.Sequential(
                nn.Linear(dim_in, dim_in),
                nn.ReLU(inplace=True),
                nn.Linear(dim_in, feat_dim)
            )

    def forward(self,X):
        X = self.head(X)
        X = F.normalize(X, dim=1)
        X = X.unsqueeze(1)
        return X

### 2. calling function

In [60]:
del mdl
torch.cuda.empty_cache()

NameError: name 'mdl' is not defined

In [17]:
assert False

AssertionError: 

# Test

In [1]:
from contrastive_learning import prepare_dataloader_pairdata

In [1]:
from data.process_data import resampling
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

### 0. shortcut

In [10]:

s = np.array([1,2,3]).reshape(-1,1)
g = np.array([2,3,1]).reshape(-1,1)
f1_score(s,g,average='micro')

0.0

### 1. Data

In [1]:
from contrastive_learning import *

# random seed
np.random.seed(1024)
torch.manual_seed(1024)

# gpu setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
parallel = True
num_workers = 0

In [2]:
X1,X2,y = import_pair_data(dirc)
X1 = X1.reshape(*X1.shape,1).transpose(0,3,1,2)
X2 = X2.reshape(*X2.shape,1).transpose(0,3,1,2)
X = np.concatenate((X1,X2),axis=1)
y,lb = label_encode(y)
y = np.concatenate((y.reshape(-1,1),y.reshape(-1,1)),axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train = reshape_axis1(X_train)
y_train = y_train.reshape(-1)
X_test = reshape_axis1(X_test)
y_test = y_test.reshape(-1)

Importing Data >>>>>> Complete


In [42]:
train_size = 1
X1 = X_train[::2]
X2 = X_train[1::2]

X_ft,y_ft = X_train,y_train
# X_ft, _, y_ft, _ = train_test_split(X_ft,y_ft, test_size=1-train_size, random_state=42)
X_ft,  y_ft ,_ = resampling(X_ft,y_ft,y_ft,oversampling=True)

X_val, y_val = X_test,y_test
X_val, y_val,_ = resampling(X_val, y_val,y_val,oversampling=False)

In [43]:
def return_dataloaders(X1,X2,X_train,y_train,X_test,y_test):    
    pretraindataset = TensorDataset(Tensor(X1),Tensor(X2))
    finetunedataset = TensorDataset(Tensor(X_train),Tensor(y_train).long())
    validatndataset = TensorDataset(Tensor(X_test), Tensor(y_test).long())
    pretrain_loader = DataLoader(pretraindataset, batch_size=bsz, shuffle=True, num_workers=num_workers, drop_last=True)
    finetune_loader = DataLoader(finetunedataset, batch_size=bsz, shuffle=True, num_workers=num_workers, drop_last=True)
    validatn_loader = DataLoader(validatndataset, batch_size=2000, shuffle=True, num_workers=num_workers)
    return pretrain_loader, finetune_loader, validatn_loader

pretrain_loader, finetune_loader, validatn_loader = return_dataloaders(X1,X2,X_ft,y_ft,X_val,y_val)
del X1,X2,X_ft,y_ft,X_val,y_val

In [7]:
pretrain_model = create_pretrain_model()
criterion = create_criterion()
optimizer = create_optimizer('pretrain',pretrain_model)
# Pretraining
pretrain_model, record = pretrain(pretrain_model,pretrain_loader,optimizer,criterion,pre_train_epochs,start=1,parallel=parallel)
record_log('pretrain',pre_train_epochs,record)
# save('pretrain',pretrain_model,optimizer,pre_train_epochs)
del criterion, record, pretrain_loader

GPU
Start Training
Epoch 1: >>>>>>>> loss: 4.799275875091553 
Epoch 2: >>>>>>>> loss: 4.774807453155518 
Epoch 3: >>>>>>>> loss: 4.667858600616455 
Epoch 4: >>>>>>>> loss: 4.521354675292969 
Epoch 5: >>>>>>>> loss: 4.423340320587158 
Epoch 6: >>>>>>>> loss: 4.072502613067627 
Epoch 7: >>>>>>>> loss: 4.111237525939941 
Epoch 8: >>>>>>>> loss: 4.119112968444824 
Epoch 9: >>>>>>>> loss: 3.7600536346435547 
Epoch 10: >>>>>>>> loss: 3.8061299324035645 
Epoch 11: >>>>>>>> loss: 3.967787742614746 
Epoch 12: >>>>>>>> loss: 4.2065863609313965 
Epoch 13: >>>>>>>> loss: 3.8210484981536865 
Epoch 14: >>>>>>>> loss: 3.9522910118103027 
Epoch 15: >>>>>>>> loss: 3.9658806324005127 
Epoch 16: >>>>>>>> loss: 3.712909698486328 
Epoch 17: >>>>>>>> loss: 3.6867988109588623 
Epoch 18: >>>>>>>> loss: 3.525918483734131 
Epoch 19: >>>>>>>> loss: 4.125702381134033 
Epoch 20: >>>>>>>> loss: 3.5361039638519287 
Epoch 21: >>>>>>>> loss: 4.015984058380127 
Epoch 22: >>>>>>>> loss: 3.5061488151550293 
Epoch 23: >>>

Epoch 183: >>>>>>>> loss: 1.211543083190918 
Epoch 184: >>>>>>>> loss: 1.287408709526062 
Epoch 185: >>>>>>>> loss: 1.2433165311813354 
Epoch 186: >>>>>>>> loss: 1.2181757688522339 
Epoch 187: >>>>>>>> loss: 0.9712308645248413 
Epoch 188: >>>>>>>> loss: 1.4181827306747437 
Epoch 189: >>>>>>>> loss: 1.4282418489456177 
Epoch 190: >>>>>>>> loss: 0.968192458152771 
Epoch 191: >>>>>>>> loss: 1.3148810863494873 
Epoch 192: >>>>>>>> loss: 1.1824588775634766 
Epoch 193: >>>>>>>> loss: 1.22581148147583 
Epoch 194: >>>>>>>> loss: 1.2566447257995605 
Epoch 195: >>>>>>>> loss: 1.0359747409820557 
Epoch 196: >>>>>>>> loss: 1.0753883123397827 
Epoch 197: >>>>>>>> loss: 1.2120661735534668 
Epoch 198: >>>>>>>> loss: 1.1246135234832764 
Epoch 199: >>>>>>>> loss: 0.9187008142471313 
Epoch 200: >>>>>>>> loss: 1.1989951133728027 
Epoch 201: >>>>>>>> loss: 1.1965751647949219 
Epoch 202: >>>>>>>> loss: 0.6409493088722229 
Epoch 203: >>>>>>>> loss: 0.7230335474014282 
Epoch 204: >>>>>>>> loss: 1.27889895439

Epoch 360: >>>>>>>> loss: 0.1334790289402008 
Epoch 361: >>>>>>>> loss: 0.12187515944242477 
Epoch 362: >>>>>>>> loss: 0.11860630661249161 
Epoch 363: >>>>>>>> loss: 0.12159059941768646 
Epoch 364: >>>>>>>> loss: 0.13246895372867584 
Epoch 365: >>>>>>>> loss: 0.19694951176643372 
Epoch 366: >>>>>>>> loss: 0.12677901983261108 
Epoch 367: >>>>>>>> loss: 0.18830709159374237 
Epoch 368: >>>>>>>> loss: 0.1414145827293396 
Epoch 369: >>>>>>>> loss: 0.19686545431613922 
Epoch 370: >>>>>>>> loss: 0.11245917528867722 
Epoch 371: >>>>>>>> loss: 0.13040699064731598 
Epoch 372: >>>>>>>> loss: 0.13630512356758118 
Epoch 373: >>>>>>>> loss: 0.16538512706756592 
Epoch 374: >>>>>>>> loss: 0.15730419754981995 
Epoch 375: >>>>>>>> loss: 0.11816998571157455 
Epoch 376: >>>>>>>> loss: 0.10370182245969772 
Epoch 377: >>>>>>>> loss: 0.1481773853302002 
Epoch 378: >>>>>>>> loss: 0.11056411266326904 
Epoch 379: >>>>>>>> loss: 0.13711601495742798 
Epoch 380: >>>>>>>> loss: 0.1515670120716095 
Epoch 381: >>>>>>

Epoch 535: >>>>>>>> loss: 0.05606852099299431 
Epoch 536: >>>>>>>> loss: 0.06799735128879547 
Epoch 537: >>>>>>>> loss: 0.067556232213974 
Epoch 538: >>>>>>>> loss: 0.0800444707274437 
Epoch 539: >>>>>>>> loss: 0.05732567608356476 
Epoch 540: >>>>>>>> loss: 0.058839134871959686 
Epoch 541: >>>>>>>> loss: 0.06276652216911316 
Epoch 542: >>>>>>>> loss: 0.06418012827634811 
Epoch 543: >>>>>>>> loss: 0.07227899134159088 
Epoch 544: >>>>>>>> loss: 0.0625920295715332 
Epoch 545: >>>>>>>> loss: 0.08055821806192398 
Epoch 546: >>>>>>>> loss: 0.05636979639530182 
Epoch 547: >>>>>>>> loss: 0.06168225780129433 
Epoch 548: >>>>>>>> loss: 0.04928453266620636 
Epoch 549: >>>>>>>> loss: 0.06913109123706818 
Epoch 550: >>>>>>>> loss: 0.07875341176986694 
Epoch 551: >>>>>>>> loss: 0.07422405481338501 
Epoch 552: >>>>>>>> loss: 0.07786382734775543 
Epoch 553: >>>>>>>> loss: 0.08879684656858444 
Epoch 554: >>>>>>>> loss: 0.04829913377761841 
Epoch 555: >>>>>>>> loss: 0.058163680136203766 
Epoch 556: >>>>

Epoch 709: >>>>>>>> loss: 0.040246929973363876 
Epoch 710: >>>>>>>> loss: 0.047778766602277756 
Epoch 711: >>>>>>>> loss: 0.04038891941308975 
Epoch 712: >>>>>>>> loss: 0.03925016522407532 
Epoch 713: >>>>>>>> loss: 0.04904984310269356 
Epoch 714: >>>>>>>> loss: 0.03704322874546051 
Epoch 715: >>>>>>>> loss: 0.04107850044965744 
Epoch 716: >>>>>>>> loss: 0.04289526492357254 
Epoch 717: >>>>>>>> loss: 0.039781585335731506 
Epoch 718: >>>>>>>> loss: 0.04943224415183067 
Epoch 719: >>>>>>>> loss: 0.03687652200460434 
Epoch 720: >>>>>>>> loss: 0.04117364063858986 
Epoch 721: >>>>>>>> loss: 0.04005522280931473 
Epoch 722: >>>>>>>> loss: 0.05183105543255806 
Epoch 723: >>>>>>>> loss: 0.03944901004433632 
Epoch 724: >>>>>>>> loss: 0.04658878222107887 
Epoch 725: >>>>>>>> loss: 0.045563921332359314 
Epoch 726: >>>>>>>> loss: 0.03927811607718468 
Epoch 727: >>>>>>>> loss: 0.04336303845047951 
Epoch 728: >>>>>>>> loss: 0.038498882204294205 
Epoch 729: >>>>>>>> loss: 0.05378075689077377 
Epoch 73

In [9]:
from train import train as finetuning

In [34]:
del finetune_model

In [45]:
1

1

In [46]:
# Fine-tuning
finetune_model = create_finetune_model(None)
criterion = nn.CrossEntropyLoss()
optimizer = create_optimizer('finetuning',finetune_model)
finetune_model, record = finetuning(finetune_model , finetune_loader, criterion, optimizer, 300, 1, validatn_loader, parallel)
cmtx,cls = evaluation(finetune_model,validatn_loader,label_encoder=lb)
record_log('finetuning',fine_tune_epochs,record,cmtx,cls)
# save('finetuning',finetune_model,optimizer,fine_tune_epochs)
del finetune_model

GPU
Start Training
Epoch 1: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.7969496250152588 
Epoch 2: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.7903813123703003 
Epoch 3: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.6142935752868652 
Epoch 4: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.309948444366455 
Epoch 5: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.1319386959075928 
Epoch 6: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.220114827156067 
Epoch 7: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 0.9855860471725464 
Epoch 8: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 0.7564971446990967 
Epoch 9: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 0.7271001935005188 
Epoch 10: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.008368730545044  accuracy: 0.5504232804232804
Epoch 11: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 0.5714215636253357 
Epoch 12: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 0.7366707921028137 
Epoch 13: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 0.40883442759513855 
Epoch 14: >>>>>>>>>>>>>>>>>

Epoch 107: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.3038486201821797e-07 
Epoch 108: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.6055280411819695e-06 
Epoch 109: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 2.197827825511922e-06 
Epoch 110: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.1484286915219855e-05  accuracy: 0.6090203487901698
Epoch 111: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.0617068824103626e-07 
Epoch 112: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 8.214109357140842e-07 
Epoch 113: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.5012576568551594e-06 
Epoch 114: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.5367557352874428e-05 
Epoch 115: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 7.282854426193808e-07 
Epoch 116: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 4.34807370766066e-05 
Epoch 117: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 7.133845656426274e-07 
Epoch 118: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.365264779451536e-06 
Epoch 119: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 7

Epoch 211: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.0784357300508418e-06 
Epoch 212: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.8626450382086546e-09 
Epoch 213: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 9.872008632783036e-08 
Epoch 214: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 7.450578820566989e-09 
Epoch 215: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 4.6566100309064495e-08 
Epoch 216: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 7.450562833355434e-08 
Epoch 217: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 6.519248074710049e-08 
Epoch 218: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.8626450382086546e-09 
Epoch 219: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.236747607435973e-06 
Epoch 220: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 6.89177710455624e-08  accuracy: 0.5932507784569254
Epoch 221: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 9.313223969797946e-09 
Epoch 222: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.8626450382086546e-09 
Epoch 223: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 3.

In [11]:
import os
import sys
from time import gmtime, strftime
import numpy as np
import pandas as pd
import seaborn as sns  # for heatmaps
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor, nn
from torch.nn import functional as F

In [15]:
def finetuning(model, train_loader, criterion, optimizer, end, start = 1, test_loader = None, parallel = None, **kwargs):

    # Check device setting
    if parallel == True:
        print('GPU')
        model = model.cuda()
    else:
        print('CPU')

    print('Start Training')
    record = {'train':[],'validation':[]}
    i = start
    #Loop
    while i <= end:
        print(f"Epoch {i}: ", end='')
        for b, (X_train, y_train) in enumerate(train_loader):

            if parallel == True:
                X_train = X_train.cuda() #.to(device)

            print(f">", end='')
            optimizer.zero_grad()
            y_pred = model(X_train)

            if parallel == True:
                X_train = X_train.cpu()
                del X_train
                y_train = y_train.cuda()

            loss   = criterion(y_pred, y_train)
            loss.backward()
            optimizer.step()

            if parallel == True:
                y_pred = y_pred.cpu()
                y_train = y_train.cpu()
                del y_pred,y_train

        # One epoch completed
        loss = loss.tolist()
        record['train'].append(loss)
        print(f' loss: {loss} ',end='')
        if (test_loader != None) and i%10 ==0 :
            acc = short_evaluation(model,test_loader,parallel)
            record['validation'].append(acc)
            print(f' accuracy: {acc}')
        else:
            print('')
        i += 1

    model = model.cpu()
    return model, record



def short_evaluation(model,test_loader,parallel):
    # copy the model to cpu
    if parallel == True:
        model = model.cpu()
    with torch.no_grad():
        for X_test, y_test in test_loader:
            y_val = model(X_test)
            predicted = torch.max(y_val, 1)[1]
            acc = f1_score(y_test.view(-1), predicted.view(-1),average='weighted')
            # acc = accuracy_score(y_test.view(-1), predicted.view(-1))
    # send model back to gpu
    if parallel == True:
        model = model.cuda()
    return acc